In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.client import device_lib
for device in device_lib.list_local_devices(): print(device.name)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

letters = 'abcdefghjklmnop'

def make_number(string):
    number = letters.find(string[0]) * 15 + int(string[1:]) - 1
    return number

def make_string(number):
    string = letters[number // 15] + str((number % 15) + 1)
    return string

def generator(games, batch_size = 64):
    while(1):
        count = 0
        number = 0
        positions = []
        moves = []
        for game in games:
            position = [0] * 225
            game = game.split(" ")
            if (game[0] == 'black'):
                for i in range(1, len(game)):
                    move = make_number(game[i])
                    if (i % 2) == 1:
                        positions.append(position.copy())
                        moves.append(move)
                        position[move] = 1
                        count += 1
                    else:
                        position[move] = -1
                    if (count == batch_size):
                        yield np.array(positions).reshape((batch_size, 15, 15, 1)), keras.utils.to_categorical(moves, 225)
                        number += 1
                        count = 0;
                        positions = []
                        moves = []
            elif (game[0] == 'white'):
                for i in range(1, len(game)):
                    move = make_number(game[i])
                    if (i % 2) == 0:
                        positions.append(position.copy())
                        moves.append(move)
                        position[move] = -1
                        count += 1
                    else:
                        position[move] = 1
                    if (count == batch_size):
                        yield np.array(positions).reshape((batch_size, 15, 15, 1)), keras.utils.to_categorical(moves, 225)
                        number += 1
                        count = 0;
                        positions = []
                        moves = []

data1 = np.array(pd.read_csv('train-1.renju'))
#data2 = np.array(pd.read_csv('train-2.renju'))
data = data1[:, 0]

C:\Users\777\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/device:CPU:0
/device:GPU:0


Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras.optimizers import Nadam
from keras.layers import BatchNormalization, Convolution2D, Input, merge, LeakyReLU
from keras.layers.core import Activation, Layer

def conv_block_3(feat_maps_out, prev):
    prev = Conv2D(feat_maps_out, (3, 3), padding='same')(prev) 
    prev = LeakyReLU()(prev)
    prev = Conv2D(feat_maps_out, (2, 2), padding='same')(prev) 
    prev = LeakyReLU()(prev)
    prev = BatchNormalization(axis=3)(prev)
    return prev

def conv_block_5(feat_maps_out, prev):
    prev = Conv2D(feat_maps_out, (5, 5), padding='same')(prev) 
    prev = LeakyReLU()(prev)
    prev = Conv2D(feat_maps_out, (3, 3), padding='same')(prev) 
    prev = LeakyReLU()(prev)
    prev = BatchNormalization(axis=3)(prev)
    return prev

def skip_block(feat_maps_in, feat_maps_out, prev):
    if feat_maps_in != feat_maps_out:
        prev = Conv2D(feat_maps_out, (1, 1), padding='same')(prev)
    return prev 

def Residual_3(feat_maps_in, feat_maps_out, prev_layer):
    skip = skip_block(feat_maps_in, feat_maps_out, prev_layer)
    conv = conv_block_3(feat_maps_out, prev_layer)
    return merge([skip, conv], mode='sum')

def Residual_5(feat_maps_in, feat_maps_out, prev_layer):
    skip = skip_block(feat_maps_in, feat_maps_out, prev_layer)
    conv = conv_block_5(feat_maps_out, prev_layer)
    return merge([skip, conv], mode='sum')

img_rows = 15 
img_cols = 15 

inp = Input((img_rows, img_cols, 1))
cnv1 = Conv2D(64, (3, 3), padding='same')(inp)
cnv1 = LeakyReLU()(cnv1)
cnv1 = BatchNormalization(axis=3)(cnv1)
r1 = Residual_3(64, 64, cnv1)
r2 = Residual_3(64, 128, r1)
r3 = Residual_3(128, 128, r2)
r4 = Residual_3(128, 256, r3)
r5 = Residual_3(256, 256, r4)
cnv2 = Conv2D(1, (1, 1), padding='same')(r5)
cnv2 = LeakyReLU()(cnv2)
cnv2 = Flatten()(cnv2)
out = Activation('softmax')(cnv2)
model = Model(input=inp, output=out)
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

C:\Users\777\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\777\Anaconda3\envs\tensorflow\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15, 15, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 15, 15, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 15, 15, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 15, 15, 64)   256         leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
conv2d_2 (

C:\Users\777\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`


In [ ]:
model.fit_generator(generator(data), steps_per_epoch = 36000000 / 64, epochs = 1)
model.save('model-2.h5')

Epoch 1/1
 25625/562500 [>.............................] - ETA: 5:31:36 - loss: 1.9777 - acc: 0.4384